### Author
Junseong Ahn
### Laboratory 
 Perception, Control, Cognition Lab : Brigham Young University

In [60]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

from skimage.morphology import disk
from skimage import data
from skimage.filters import rank

from scipy import ndimage as ndi
import time

# Set up the path for this demo

In [2]:
#video path
video_path = 'IMG_4296.MOV'
#basename for images from video  
img_name = 'image'
#path of saved images  
img_path = 'images_resized'
#frame rate for saving video 
frameRate = 0.1


In [3]:
save_img_path = os.path.join(img_path, img_name)


# Background-Subtraction with moving scene

Background Subtraction is usually used for stationary scene, however, with an assumption, the background is plain, we can use the technique for a moving scene video. This is my work of background subtraction algrotihm for a moving scene. Note that this jupter notebook is analyzing just one of videos (IMG_4296.MOV) since we can use normal background suppression techniques for the second video(orcalab-rubbing-beach-underwater.mp4).

Also, Note that this notebook is the half of all the algorithm procedures (what was in the presentation).

In [4]:
def video2Image(video_path, save_path, frameRate):
    """
    change video to images and save to save_path with frameRate.
    Param:
        video_path (str) : path of video
        save_path : save path of image
        frameRate : the frame of video to save
    """
    vidcap = cv2.VideoCapture(name_video)
    def getFrame(sec):
        vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
        hasFrames,image = vidcap.read()
        if hasFrames:
            image = img = cv2.resize(image, (int(image.shape[1]), int(image.shape[0])))
            cv2.imwrite(save_path+str(count)+".jpg", image)     # save frame as JPG file
        return hasFrames
    
    sec =0
    count=1
    success = getFrame(sec)
    start = time.time()
    while success:
        count = count + 1
        sec = sec + frameRate
        sec = round(sec, 2)
        success = getFrame(sec)


In [5]:
def getImages(img_name, img_path, n_img):
    """
    Get color images(.jpg) from img_path. 
    Note that it is the order of the images should be next to img_name.(e.g tree13)
    """
    image_names = [img_name+str(i)+".jpg" for i in range(n_img)]
    imgs = [cv2.imread(os.path.join(img_path,image_names[i]), cv2.IMREAD_GRAYSCALE) for i in range(1,len(image_names))]
    imgs_color = [cv2.imread(os.path.join(img_path,image_names[i]), cv2.COLOR_BGR2RGB) for i in range(1,len(image_names))]

    return imgs, imgs_color

In [6]:
imgs, imgs_color = getImages(img_name, img_path, 250)

## normal background suppression using open-cv 

In [61]:
cap = cv2.VideoCapture(0)
fgbg = cv2.bgsegm.createBackgroundSubtractorMOG()
fgbgAdaptiveGaussain = cv2.createBackgroundSubtractorMOG2()
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
fgbgBayesianSegmentation = cv2.bgsegm.createBackgroundSubtractorGMG()


for i in range(len(imgs)):
    fgmask = fgbg.apply(imgs[i])
    fgbgAdaptiveGaussainmask = fgbgAdaptiveGaussain.apply(imgs[i])
    fgbgBayesianSegmentationmask = fgbgBayesianSegmentation.apply(imgs[i])
    fgbgBayesianSegmentationmask = cv2.morphologyEx(fgbgBayesianSegmentationmask,cv2.MORPH_OPEN,kernel)

    cv2.namedWindow('Background Subtraction Bayesian Segmentation',0)
    cv2.namedWindow('Background Subtraction',0)
    cv2.namedWindow('Background Subtraction Adaptive Gaussian',0)
    cv2.namedWindow('Original',0)

    cv2.resizeWindow('Original', 450,450)
    cv2.imshow('Background Subtraction Bayesian Segmentation',fgbgBayesianSegmentationmask)
    cv2.imshow('Background Subtraction',fgmask)
    cv2.imshow('Background Subtraction Adaptive Gaussian',fgbgAdaptiveGaussainmask)
    cv2.imshow('Original',imgs[i])

    k = cv2.waitKey(1) & 0xff

    if k==ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Analysis of the result

As you can see, because the camera was moving, the background aglorithm can't differeniate the background(the mountain) and the ship; therefore, I propose background suppression using 3D morpholocal Gradient

# 3D morhpholocal Gradient

In [43]:
class BackgroundSuppression():
    """
    Dynaimcally calculate gradient flows to simulate real video situation.
    """
    def __init__(self, imgs, imgs_color, k, kernel_size):
        if len(imgs) < k:
            raise ValueError("you need at least %d number of images"%d)
        self.img_size = imgs[0].shape
        self.imgs = [cv2.GaussianBlur(imgs[i],(5,5),cv2.BORDER_DEFAULT) for i in range(len(imgs))]
        self.imgs_color = imgs_color
        self.k = k
        self.kernel = np.ones((kernel_size,kernel_size),np.uint8)
        
        self.dilations = np.array([cv2.dilate(self.imgs[i], self.kernel,iterations=1) for i in range(k-1)])
        self.erosions = np.array([cv2.erode(self.imgs[i], self.kernel,iterations=1) for i in range(k-1)])
        
        self.gradients = self.dilations - self.erosions
        self.gradients3d = [self.calculate3dGradientVectorize(i, 0, stability=False, init=True) for i in range(0,k-1)]            
            
    def update(self, img):
        blur_img = cv2.GaussianBlur(img,(5,5),cv2.BORDER_DEFAULT)
        dilation, erosion = np.expand_dims(cv2.dilate(blur_img, self.kernel,iterations=1), axis=0),\
                            np.expand_dims(cv2.erode(blur_img, self.kernel,iterations=1), axis=0)
        gradient = dilation - erosion
        self.dilations = np.concatenate((self.dilations, dilation),axis=0)
        self.erosions = np.concatenate((self.erosions,erosion),axis=0)
        self.gradients = np.concatenate((self.gradients,gradient),axis=0)

    
    def calculate3dGradientVectorize(self, inx, weights=1, stability=True, init=False):
        """
        Calculate gradient in the time interval t. if weights is not zero, we don't have any speical attention to current scene
        it is for stability option of background.
        """
        if init:
            dilation3d = np.max(self.dilations[0:inx+1], axis=0)
            erosion3d = np.min(self.erosions[0:inx+1], axis=0)
            #get gradient standard deviation if it is first frame, diff is 1
            if inx == 0:
                diff = np.ones_like(self.gradients[0])
            else:
                diff = np.std(self.gradients[0:inx+1], axis=0)
        else:
            dilation3d = np.max(self.dilations[-self.k:], axis=0)
            erosion3d = np.min(self.erosions[-self.k:], axis=0)
            #get gradient standard deviation
            diff = np.std(self.gradients[-self.k:].copy(), axis=0)

        
        if stability:
            prev_gradients = self.gradients3d[-(k+1):]
            current_grad3d = dilation3d - erosion3d
            #if weights are not zero, we have more attention/less attention to current grad
            if weight != 0:
                current_grad3d += weight*current_grad3d
            #get mean of gradient in k time interval
            grad3d = np.mean(prev_gradients.extend(current_grad3d), axis=0)
        else:
            grad3d = dilation3d - erosion3d
        


        #for normalize std, we divide by max of diff
        grad3d = grad3d * diff / np.max(diff)
        return grad3d
    
    def showImages(self, option):
        """
        params
        options (int):
            1. 3d grad
            2. image flow with 3dgrad
        """

        prvs = self.gradients3d[0]
        if option == 1:
            hsv = np.zeros_like(imgs[0])
        else:
            hsv = np.zeros_like(imgs_color[0])
        hsv[...,1] = 255

        alpha = 6
        
        print(len(self.imgs))
        for i in range(self.k-1, len(self.imgs)):            
            self.update(imgs[i])
            self.gradients3d.append(self.calculate3dGradientVectorize(i, stability=False))
            
            if option == 1:
                #the reason we thresh holding value with the gradient mean times alpha is 
                # there are a lot more background and the mean is skewed; there should be non-hard coding way to find using std
                frame = (self.gradients3d[i]>alpha*self.gradients3d[i].mean()).astype(np.uint8)*255
                frame_name = "3d grad"
            elif option == 2:
                #calculate optimal flow between two gradients with Lucas–Kanade method
                # https://en.wikipedia.org/wiki/Lucas%E2%80%93Kanade_method

                flow = cv2.calcOpticalFlowFarneback(self.gradients3d[i-1],self.gradients3d[i], None, 0.5, 3, 15, 3, 5, 1.2, 0)
                mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
                hsv[...,0] = ang*180/np.pi/2
                hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
                rgb = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)
                frame = imgs_color[i]+rgb
#                 print(flow.shape)
#                 asdf
                frame_name = "Optical flow: directional gradient of 3d Morphological gradient"
            cv2.imshow(frame_name,frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        # Release everything if job is finished
        cap.release()
        cv2.destroyAllWindows()

In [44]:
backgroundsuppression = BackgroundSuppression(imgs, imgs_color, 3, 7)

In [46]:
backgroundsuppression.showImages(2)

249


# Note: What is alpha?

As you can see there is hyperparamter alpha when we are threshholding low gradient. The reason that I have mean*alpha is that there are signicantly more background than the foreground. Therefore, the mean is skewed. I think there is a way to find this parameter rather than hard coding it (e.g. check skewness of gradients) 

In addiiton, in the presentation, Dr. Mukherjee suggests that we can use some transformation to remove the gradient for background; however, I need to think about it more.




# Note: why does my background supression method is getting slower after few seconds?

When we run %prun backgroundsuppression.showImages(1), to process 249 images, it takes 28 second without any optimization (0.1second per frame). However, if I profile this, I can make it much faster.


For example, actually I don't need to use np concatenate (by saving the only interval arrays), it takes most of time (20 seconds, for more information please, look at this link  <a href="https://drive.google.com/open?id=12147WdU7pQY-jTxgpzwpeaRVkoQ-JxHE">Time Per Line</a>), which means we can change 28 second to almost 8 second(0.03 second per frame).



backgroundsuppression.showImages(1)

In [25]:
backgroundsuppression.showImages(2)

249


error: OpenCV(4.2.0) /io/opencv/modules/core/src/array.cpp:2492: error: (-206:Bad flag (parameter or structure field)) Unrecognized or unsupported array type in function 'cvGetMat'


# How do I interperate backgroundsuppression.showImages(2)?

Meaning of flow Markers
<ul>
    <li>The intensity of color: the velocity of pixel</li>
    <li>The kind of color: the direction of pixel</li>
</ul>

# Reference
<ul>
        <li>Gandhi, T. L. (1999). Image sequence analysis for object detection and segmentation.</li>
    <li>Pardàs, M., & Salembier, P. (1994). 3D morphological segmentation and motion estimation for image sequences. Signal processing, 38(1), 31-43.</li>
    <li>Stauffer, C., & Grimson, W. E. L. (2000). Learning patterns of activity using real-time tracking. IEEE Transactions on pattern analysis and machine intelligence, 22(8), 747-757.</li>
    <li>Mukherjee, D., Wu, Q. J., & Nguyen, T. M. (2013). Gaussian mixture model with advanced distance measure based on support weights and histogram of gradients for background suppression. IEEE Transactions on Industrial Informatics, 10(2), 1086-1096.</li>
</ul>
